In [129]:
import pandas as pd
import re
import numpy as np
import os

# For R mixed effects models
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri

In [130]:
os.getcwd()

'/Users/nickbachelder/Desktop/Personal Code/Kaggle/Madness'

In [131]:
## Get tourney seed data

mseed = pd.read_csv("data/MNCAATourneySeeds.csv")
mseed['M_W'] = 'M'
wseed = pd.read_csv("data/WNCAATourneySeeds.csv")
wseed['M_W'] = 'W'
seed = pd.concat([mseed, wseed])
seed['ReigonSeed'] = seed.Seed
seed['Seed'] = [seeds[1:] for seeds in seed.Seed]
seed['play_in_seed'] = list(map(lambda x : x[-1].isalpha(), seed.Seed))
seed['Seed'] = list(map(lambda x : int(re.sub('\D', "", x)), seed.Seed))
seed.to_csv('data/tourney_seeds.csv', index=False) 

mslot = pd.read_csv("data/MNCAATourneySlots.csv")
mslot['M_W'] = 'M'
wslot = pd.read_csv("data/WNCAATourneySlots.csv")
wslot['M_W'] = 'W'
slot = pd.concat([mslot, wslot])
conditions = [
    ["R1" in slot for slot in slot.Slot],
    ["R2" in slot for slot in slot.Slot],
    ["R3" in slot for slot in slot.Slot],
    ["R4" in slot for slot in slot.Slot],
    ["R5" in slot for slot in slot.Slot],
    ["R6" in slot for slot in slot.Slot],
]
choices = [1,2,3,4,5,6]
slot["GameRound"] = np.select(conditions, choices, default = 0)
slot
slot.to_csv('data/tourney_slots.csv', index=False) 


In [133]:
m_season_details = pd.read_csv("data/MRegularSeasonDetailedResults.csv")
m_season_details['M_W'] = 'M'
w_season_details = pd.read_csv("data/WRegularSeasonDetailedResults.csv")
w_season_details['M_W'] = 'W'
season_details = pd.concat([m_season_details, w_season_details])
season_details


winner_season_details = season_details[['M_W', "Season", "DayNum", "WTeamID", "LTeamID", "WScore", "LScore", "WLoc", "WFGM", 
'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', "NumOT"
]].set_axis(['M_W', "Season", "DayNum", "TeamID", "OppTeamID", "Score", "OppScore", "Loc", "FGM", 
'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR',
       'Ast', 'TO', 'Stl', 'Blk', 'PF', "NumOT"], axis=1, inplace=False)
winner_season_details['Result'] = 1
       
loser_season_details = season_details[['M_W', "Season", "DayNum", "WTeamID", "LTeamID", "WScore", "LScore", 
"WLoc", 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', "NumOT"
]].set_axis(['M_W', "Season", "DayNum", "OppTeamID", "TeamID", "OppScore", "Score", "Loc", "FGM", 
'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR',
       'Ast', 'TO', 'Stl', 'Blk', 'PF', "NumOT"], axis=1, inplace=False)
conditions = [(loser_season_details.Loc == "H"), 
(loser_season_details.Loc == "A"), 
(loser_season_details.Loc == "N")]
choices = ["A", "H", "N"]
loser_season_details['Result'] = 0
loser_season_details["Loc"] = np.select(conditions, choices)
season_details = pd.concat([winner_season_details, loser_season_details])
season_details['mov'] = season_details.Score - season_details.OppScore
season_details.to_csv('data/game_results.csv', index=False) 

In [135]:
m_tourney_results = pd.read_csv("data/MNCAATourneyCompactResults.csv")
m_tourney_results['M_W'] = 'M'
w_tourney_results = pd.read_csv("data/WNCAATourneyCompactResults.csv")
w_tourney_results['M_W'] = 'W'
tourney_details = pd.concat([m_tourney_results, w_tourney_results])
tourney_details

tourney_details.to_csv('data/tourney_results.csv', index=False) 

In [136]:
mteams = pd.read_csv("data/MTeams.csv")
mteams['M_W'] = 'M'
wteams = pd.read_csv("data/WTeams.csv")
wteams['M_W'] = 'W'
teams = pd.concat([mteams, wteams])
teams.to_csv('data/teams.csv', index=False) 

In [137]:
# Get mixed effects for each team for each season
season_game_results = pd.read_csv("data/game_results.csv").query("Season >= 2003")
all_effects = []
# Load necessary R packages
base = importr('base')
stats = importr('stats')
lme4 = importr('lme4')
pandas2ri.activate()
for season in range(2010, max(season_game_results.Season)+1):
    print(season)
    print("--------")
    for m_w in ["M", "W"]:
        print(m_w)
        # Build own rankings with mixed effects model
        mod_df = season_game_results.query("Season == @season & M_W == @m_w & NumOT == 0")[["TeamID", "OppTeamID", "Result"]].copy()
        if len(mod_df) == 0:
            continue
        mod_df['TeamID']=mod_df['TeamID'].astype("str")
        mod_df['OppTeamID']=mod_df['OppTeamID'].astype("str")
        # Convert filtered data to R dataframe
        robjects.globalenv['filtered_data'] = pandas2ri.py2rpy(mod_df)
        # Define the R model formula
        formula = robjects.Formula("Result ~ 1 + (1 | TeamID) + (1 | OppTeamID)")

        # Fit the model using lme4
        model = lme4.glmer(formula=formula, data=robjects.globalenv['filtered_data'], family="binomial")

        # Get coefs
        quality_df_r=base.data_frame(lme4.random_effects(model).rx2("TeamID"))
        quality_df_python=pandas2ri.rpy2py_dataframe(quality_df_r).reset_index().rename(columns={'index': 'TeamID', 'X.Intercept.' : 'Effect'})
        quality_df_python["Season"] = season
        quality_df_python["M_W"] = m_w
        all_effects.append(quality_df_python)
all_effects = pd.concat(all_effects, axis = 0)
all_effects["TeamID"] = all_effects["TeamID"].astype(int)
all_effects.to_csv('data/all_effects.csv', index=False) 

/Users/nickbachelder/opt/anaconda3/envs/data-science/lib/python3.10/site-packages/rpy2/robjects/vectors.py:1017: UserWarning: R object inheriting from "POSIXct" but without attribute "tzone".
  warnings.warn('R object inheriting from "POSIXct" but without '


2010
--------
M
W
2011
--------
M
W
2012
--------
M
W
2013
--------
M
W
2014
--------
M
W
2015
--------
M
W
2016
--------
M
W
2017
--------
M
W
2018
--------
M
W
2019
--------
M
W
2020
--------
M
W
2021
--------
M
W
2022
--------
M
W
2023
--------
M
W
2024
--------
M
W


In [138]:
all_effects

,TeamID,Effect,Season,M_W
0,1102,-0.732372,2010,M
1,1103,0.969696,2010,M
2,1104,1.005688,2010,M
3,1105,-1.881346,2010,M
4,1106,-1.462103,2010,M
...,...,...,...,...
355,3474,-2.768385,2024,W
356,3475,0.445063,2024,W
357,3476,-3.483012,2024,W
358,3477,-0.711723,2024,W
